In [2]:
import torch
import torchaudio
import numpy as np
import os
import pandas as pd
import json

In [2]:
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
import random
import warnings
warnings.filterwarnings('ignore')

In [3]:
import random

In [4]:
list_ = np.unique(pd.read_csv("./breath_csv_2/Breath_3.csv").iloc[:,73].values)

In [12]:
def count(list_array):
    Total = list_array
    name = []
    for i in Total:
        name.append(i[:4])
    unique = []
    before_after = []
    for i in Total:
        if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
            unique.append(i)
        else:
            before_after.append(i) 

    Asthmatic_Female = []
    Asthmatic_Male = []
    Healthy_Male = []
    Healthy_Female = []
    NAsthma = []
    before = {"male" : [], "female" : []}
    after = {"male" : [], "female" : []}
    for file in unique:
        if file.split("_")[2] == 'Asthma' or file.split("_")[2] == 'asthma' :
            if file.find("_M_")!=-1:
                Asthmatic_Male.append(file)
            if file.find("_F_")!=-1:
                Asthmatic_Female.append(file)
        if file.split("_")[2] == 'NAsthma':
            NAsthma.append(file)
        if file.find("_C_")!=-1:
            if file.find("_M_")!=-1:
                Healthy_Male.append(file)
            if file.find("_F_")!=-1:
                Healthy_Female.append(file)

    for file in before_after:
            if file.find("efore") != -1:
                if file.find("_M_") != -1 or file.find("_m_")!=-1:
                    before["male"].append(file)
                if file.find("_F_")!=-1:
                    before["female"].append(file)
            if file.find("fter")!= -1:
                if file.find("_M_") != -1 or file.find("_m_")!=-1:
                    after["male"].append(file)
                if file.find("_F_")!=-1:
                    after["female"].append(file)

    print(f"Total = {len(Total)}")
    print(f"Asthmatic_male = {len(Asthmatic_Male)+ len(before['male'])+len(after['male'])}")
    print(f"Asthmatic_female = {len(Asthmatic_Female)+ len(before['female'])+len(after['female'])}")
    print(f"Healthy_Male = {len(Healthy_Male)}")
    print(f"Healthy_female = {len(Healthy_Female)}")
    print(f"Asthmatic_male_unique = {len(Asthmatic_Male)}")
    print(f"Asthmatic_female_unique = {len(Asthmatic_Female)}")
    print(f"Before_after = {len(before_after)}")
    print(f"Before_male = {len(before['male'])}")
    print(f"Before_female = {len(before['female'])}")
    print(f"after_male = {len(after['male'])}")
    print(f"after_female = {len(after['female'])}")

    print(f"NAsthma = {len(NAsthma)}")

In [89]:
count(list_)

Total = 274
Asthmatic_male = 90
Asthmatic_female = 68
Healthy_Male = 55
Healthy_female = 51
Asthmatic_male_unique = 46
Asthmatic_female_unique = 28
Before_after = 84
Before_male = 22
Before_female = 20
after_male = 22
after_female = 20
NAsthma = 10


In [5]:
def train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after,before,after):
    random.seed(0)
    
    
    A_F = np.concatenate(([i for i in Asthmatic_Female if i.find("after") != -1 ], after["female"]))
    A_M = np.concatenate(([i for i in Asthmatic_Male if i.find("after") != -1 ], after["male"]))
    
    test_set = np.concatenate(((np.random.choice(np.array(A_M), replace=False, size=(7)),
                                np.random.choice(np.array(A_F), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Female), replace=False, size=(7)))))
    
    train_set = np.concatenate((np.array(A_M),
                                np.array(A_F),
                                np.array(Healthy_Male),
                                np.array(Healthy_Female)))
    train_set = np.setdiff1d(train_set,test_set)
    return train_set, test_set

In [11]:
def train_test_txt(sound_dataframe):
#     sound_dataframe = list_
#     Total = np.unique(sound_dataframe.iloc[:,sound_dataframe.shape[1]-1])
    Total = sound_dataframe
    name = []
    for i in Total:
        name.append(i[:4])
    unique = []
    before_after = []
    for i in Total:
        if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
            unique.append(i)
        else:
            before_after.append(i) 

    Asthmatic_Female = []
    Asthmatic_Male = []
    Healthy_Male = []
    Healthy_Female = []
    NAsthma = []
    before = {"male" : [], "female" : []}
    after = {"male" : [], "female" : []}
    for file in unique:
        if file.split("_")[2] == 'Asthma' or file.split("_")[2] == 'asthma' :
            if file.find("_M_")!=-1:
                Asthmatic_Male.append(file)
            if file.find("_F_")!=-1:
                Asthmatic_Female.append(file)
        if file.split("_")[2] == 'NAsthma':
            NAsthma.append(file)
        if file.find("_C_")!=-1:
            if file.find("_M_")!=-1:
                Healthy_Male.append(file)
            if file.find("_F_")!=-1:
                Healthy_Female.append(file)

    for file in before_after:
            if file.find("efore") != -1:
                if file.find("_M_") != -1 or file.find("_m_")!=-1:
                    before["male"].append(file)
                if file.find("_F_")!=-1:
                    before["female"].append(file)
            if file.find("fter")!= -1:
                if file.find("_M_") != -1 or file.find("_m_")!=-1:
                    after["male"].append(file)
                if file.find("_F_")!=-1:
                    after["female"].append(file)


    Train, Test = train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after,before, after)
    
    return Train, Test

In [90]:
for i in range(0,10):
    Train, Test = train_test_txt(list_)
#     print(len(Train))
    np.savetxt(f"./breath_sets_wo_after/Train/Train_{i}.txt",Train, '%s')
    np.savetxt(f"./breath_sets_wo_after/Test/Test_{i}.txt",Test, '%s')

In [91]:
count(np.concatenate((Train, Test)))

Total = 167
Asthmatic_male = 34
Asthmatic_female = 27
Healthy_Male = 55
Healthy_female = 51
Asthmatic_male_unique = 34
Asthmatic_female_unique = 27
Before_after = 0
Before_male = 0
Before_female = 0
after_male = 0
after_female = 0
NAsthma = 0


### EXP 8 

In [184]:
set_old = pd.read_csv("open_smile_database_93.txt", header= None)

In [185]:
set_new = pd.read_csv("Renamed_Akansh.txt", delimiter= "\t", header=None, )

In [186]:
set_new[set_new.loc[:,1] == set_old[0][0] + " "][5]

0    4534_C_Na_M_Na_Na_Y.wav 
Name: 5, dtype: object

In [187]:
set_old.loc[:,1] = 0

In [189]:
for i,j in enumerate(set_old.loc[:,0]):
    set_old.loc[i,1] = set_new[set_new.loc[:,1] == j + " "][5].values[0]

In [193]:
set_old

,0,1
0,2016_07_04_11_30_NaveenP_C_Na_M_Na_20_157_73_N...,4534_C_Na_M_Na_Na_Y.wav
1,2016_07_05_11_45_HaksaR_C_Na_M_Na_30_164_51_Na...,4126_C_Na_M_Na_Na_Y.wav
2,2016_07_05_12_00_VasuK_C_Na_M_Na_26_169_62_Na_...,4761_C_Na_M_Na_Na_Y.wav
3,2016_07_07_03_15_TulikaKarmakar_P_Asthma_F_bef...,4491_P_Asthma_F_before_Mil_Y.wav
4,2016_07_07_10_30_Prahalad_C_Na_M_Na_31_168_60_...,4922_C_Na_M_Na_Na_Y.wav
...,...,...
88,2019_02_13_11_55_Maheshwari_C_Na_F_Na_52_154_6...,4951_C_Na_F_Na_Na_Y.wav
89,2019_02_15_12_07_Suseelamma_C_Na_F_Na_60_159_5...,4780_C_Na_F_Na_Na_Y.wav
90,2019_02_20_12_04_GeethaK_C_Na_F_Na_42_145_65_N...,4990_C_Na_F_Na_Na_Y.wav
91,2019_02_20_12_24_AnuvaDutta_C_Na_F_Na_60_157_5...,4777_C_Na_F_Na_Na_Y.wav


In [194]:
list_ = [i[:-4]+'json' for i in set_old[1]]

In [199]:
count(list_)

Total = 93
Asthmatic_male = 26
Asthmatic_female = 19
Healthy_Male = 24
Healthy_female = 24
Asthmatic_male_unique = 26
Asthmatic_female_unique = 19
Before_after = 0
Before_male = 0
Before_female = 0
after_male = 0
after_female = 0
NAsthma = 0


In [204]:
for i in range(0,10):
    Train, Test = train_test_txt(list_)
#     print(len(Train))
    np.savetxt(f"./breath_sets_old/Train/Train_{i}.txt",Train, '%s')
    np.savetxt(f"./breath_sets_old/Test/Test_{i}.txt",Test, '%s')

### EXP 9

In [9]:
for i in range(0,10):
    Train, Test = train_test_txt(list_)
#     print(len(Train))
    np.savetxt(f"./breath_sets_w_after/Train/Train_{i}.txt",Train, '%s')
    np.savetxt(f"./breath_sets_w_after/Test/Test_{i}.txt",Test, '%s')

In [13]:
count(np.concatenate((Train, Test)))

Total = 201
Asthmatic_male = 55
Asthmatic_female = 40
Healthy_Male = 55
Healthy_female = 51
Asthmatic_male_unique = 55
Asthmatic_female_unique = 40
Before_after = 0
Before_male = 0
Before_female = 0
after_male = 0
after_female = 0
NAsthma = 0
